2023/9/21 須藤

# 10章 協働作業(コラボレーション)

Pythonには、明確なインタフェースを持ち、きちんと定義されたAPIの作成を助けてくれる言語機能があります。Python コミュニティは、時間をかけて保守性を最大化できるベスト・プラクティスを確立してきました。さまざまな環境にまたがった大人数のチームで一緒に作業するのを可能にする標 準的なツールも Python とともに使えるようになっています。

Pythonプログラムで他の人と協働するには、コードの書き方に注意する必要があります。自分だけで作業していたとしても、標準ライブラリやオープンソースパッケージを介して誰かが書いたコードを使っていることが多いでしょう。他の Python プログラマと協働するのが容易になる仕組みを理解することが必要です。


## 項目 82 コミュニティのモジュールをどこで見つけられるかを 知っておく

Python には、プログラムにインストールして利用するモジュールのためのセントラルリポジトリ (https://pypi.org) があります。ここにあるモジュールは、読者のような人々、つまり Python コミュニ ティによって作成され保守されています。 よく知らない課題に挑戦している場合、 Python パッケージインデックス (PyPI) は、目的に近づけるコードを探すために最適な場所です。

パッケージインデックスを使うには、 pip という名 (pip installs package の再帰的な頭字語とされていた)　のコマンドラインツールを使う必要があります。pip は、python3 -m pip で実行でき、パッケージがシステムの正しいPythonに確実に対応するようにします(「項目1 使用するPython のバージョンを知っておく」 参照)。 pip を使って新たなモジュールをインストールするのは簡単です。例えば、次に示すのは、別の項目で（「項目67 ローカルクロックにはtime ではなく datetime を使う」 参照）使ったpytzモジュールのインストール例です。

```
python3 -m pip install pytz
Collecting pytz

Downloading ...
Installing collected packages. pytz
Successfully installed pytz-2018.9
```

pipは組み込みモジュール venv と一緒に使うのが、プロジェクト用に一貫したパッケージを記録管理するには一番良い選択です(「項目 83 隔離された複製可能な依存関係のために仮想環境を使う」参 照)。専用のPyPIパッケージを作り、Python コミュニティと共有したり、pipで使用できる専用パッケージリポジトリをホストすることもできます。

PyPIのモジュールは、いずれも自前のソフトウェアライセンスを持っています。特によく使われる ほとんどのパッケージがフリーかオープンソースライセンス(詳細は、https://opensource.org/参照) です。ほとんどの場合、これらのライセンスの下では、プログラムにモジュールの複製を入れることができます。不明な点がある場合は、 弁護士に相談することです。

### 覚えておくこと

- Python パッケージインデックス (PyPI) には、 Python コミュニティで作られ保守されている、よく使われる豊富なパッケージがある。
- pipは、PyPIからのパッケージのインストールに使うコマンドラインツール
- PyPIのモジュールの大多数は、 フリーまたはオープンソースのソフトウェア。

## 項目 83 隔離された複製可能な依存関係のために仮想環境を使う

より大きくてより複雑なプログラムの作成においては、Python コミュニティからのさまざまなパッケージに依存するようになることがよくあります (「項目82 コミュニティのモジュールをどこで見つ けられるかを知っておく」参照)。pytz, numpy, その他多くのパッケージをインストールするためにpython3 -m pip コマンドラインを実行することも多いでしょう。

問題は、デフォルトでpipがグローバルロケーションで新たなパッケージをインストールしてしまうことです。これは、システム上のすべてのPythonプログラムが、これらのインストールしたモジュールによって影響を受けることを意味します。理論的には、これは問題とはならないはずです。パッケージをインストールしただけで import　していないのに、なぜプログラムに影響が出るのでしょうか。 問題は、推移的依存関係から発生します。インストールしたパッケージに依存しているパッケージです。例えば、pipを使ってインストールした後で、Sphinxパッケージが何に依存しているかがわかります。


```
$ python3 -m pip show Sphinx

Name: Sphinx
Version: 2.1.2
Summary: Python documentation generator
Location: /usr/local/lib/python3.8/site-pacages
Requires: alabaster, imagesize, requests,  sphinxcontrib-applehelp, sphinxcontrib-qthelp, Jinja2, setuptools, sphinxcontrib-jsmath, sphinxcontrib-serializinghtml, Pygments, snowballstemmer, packaging, sphinxcontrib-devhelp, sphinxcontrib-htmlhelp, babel, docutils
Required-by:
```

flaskのような別のモジュールをインストールすると、それもJinja2 パッケージに依存することがわかります。


```
$ python3 -m pip show flask
Name: Flask
Version: 1.0.3
Summary: A simple framework for building complex web applications.
Location: /usr/local/lib/python3.8/site-packages
Requires: itsdangerous, click, Jinja2, Werkzeug
Required-by:
```

Sphinx と flaskは、常に開発が進められているので、時間が経つと問題が生じます。現時点では、おそらくどちらもJinja2の同じバージョンを要求していて、すべて問題なしでしょう。今から、半年か1年でJinja2が新しいバージョンに移行して、ライブラリのユーザに変更を迫るかもしれません。 python3 -m pip install --upgrade Jinja2 を実行して、Jinja2のバージョンを上げた場合、 flaskは問題なく動くのに、 Sphinxは動かなくなる恐れがあります。

このような問題の原因は、 Python がある時点でインストールされたモジュールに1つのバージョンしか対応できないことにあります。インストールしたパッケージが最新バージョンを使わなければならないのに、他のパッケージは古いバージョンを使わなければならないとすると、システムが正常に働くのは無理なのです。この状況は、**依存性地獄**と呼ばれます。

このような障害は、パッケージの保守担当者が最善の努力をして、リリースしたバージョンの間で APIの互換性を維持していても起こります(「項目85 モジュールの構成にパッケージを用い、安定なAPIを提供する」参照)。ライブラリの新バージョンがAPIを使うコードが信頼している振る舞いを少し変えることがあります。システムのユーザが、あるパッケージを新バージョンに更新したが、他のはそのままにしており、そのため依存関係が壊れることがあります。十分注意していないと、基盤から揺らいでしまうというリスクが常にあるのです。

このような困難は、別のコンピュータで作業をしている他の開発者と協働しているとさらに大きくなります。彼らが自分のマシンにインストールしている Python とそのパッケージのバージョンが、自分のとは少々異なるだろうという最悪の事態を想定しておくのが安全です。この状況から、あるプログラマのマシンではコードベースが完璧に動くのに、他の人のでは動かないということが起こりうるのです。


こういった問題のすべてが、仮想環境を提供する venv と呼ばれるツールを使うことで解決できます。Python 3.4以降では、pip と venv は、Python インストール時にデフォルトで利用可能です(python -m venvでも使えます)。

venv は、隔離されたバージョンのPython環境を作ることができます。venv を使うと、同じシステムで同時期に同じパッケージの異なるバージョンを問題なく保持できます。これは、同じコンピュー タで、多数の異なるプロジェクトを行い、多数の異なるツールを使うことができることを意味します。venv は、パッケージの明示されたバージョンとその依存関係を完全に別のディレクトリ構造にインストールするので、これが可能です。これは、自分のコードが確実に動作する Python 環境を再生することも可能にします。予期しない破綻を避けるための信頼できる方法です。


### コマンドラインでvenv を使う

venv を効果的に使うための簡単なチュートリアルを行います。 このツールを使う前に、システム上で python3 コマンドラインの意味を知っておくことが重要です。著者のコンピュータでは、 python3 は /usr/local/bin ディレクトリに位置し、バージョン3.8.0です(「項目1 使用する Pythonのバー ジョンを知っておく」参照)。

```
$ which python3
/usr/local/bin/python3

$ python3-version info
Python 3.8.0
```

環境がどうセットアップされているか確認するために、pytz モジュールをインポートするコマンド を実行してもエラーが起こらないかどうか試すことができます。これがうまくいくのは、既に pytz モジュールがグローバルモジュールとしてインストールされているからです。


```
$ python3 -c 'import pytz'
$
```
※訳注:Windows環境では、二重引用符を使うのでpython3 -c "import pytz" と入力する

venv を使って、myproject という新たな仮想環境を作成しましょう。仮想環境は独自のディレクトリで動作します。コマンドの結果は、仮想環境を管理するディレクトリとファイルのツリーになります。

```
$ python3 -m venv myproject
$ cd myproject
$ ls
bin include lib pyvenv.cfg
```

仮想環境を使うために、 sourceコマンドでbin/activate シェルスクリプトを使います。 activate は、すべての環境変数を仮想環境に合致するように修正します。さらに、コマンドライン プロンプトを仮想環境名 ('myproject') を含めるように変更して、自分が何をしているかをはっきりさせてくれます。


```
$ source bin/activate
(myproject)$
```

Windows では、同じスクリプトは次のようになります。

```
C:\> myproject\Scripts\activate.bat
(myproject) C:>
```

またはPowerShell で次のようになります。

```
PS C:\> myproject\Scripts\activate.psl
(myproject)PS C:>
```

アクティベーションの後は、python3 コマンドラインツールが仮想環境ディレクトリに移動しています。

```
(myproject)$ which python3
/tmp/myproject/bin/python3
(myproject)$ ls -l /tmp/myproject/bin/python3
...-> /usr/local/bin/python3.8
```

これは、外部のシステムへの変更が仮想環境に及ばないことを保証します。外部システムがデフォルトのpython3をバージョン3.9に更新しても、仮想環境はバージョン3.8と明示された状態を保ちます。

venvで作成した仮想環境は、pip と setuptools を除いては、何のパッケージもインストールされていないところから始まります。外側のシステムでグローバルモジュールとしてインストールされていた pytz を使おうとすると、仮想環境では知られていないので失敗します。


```
(myproject)$ python3 -c 'import pytz'
Traceback (most recent call last):
 File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'pytz'
```

pipを使って、 pytzモジュールを仮想環境にインストールできます。

```
(myproject)$ python3 -m pip install pytz
Collecting pytz
  Downloading ...
Installing collected packages: pytz
Successfully installed pytz-2019.1
```

インストールしたら、同じインポートコマンドのテストで動くかどうか検証できます。

```
(myproject)$ python3 -c 'import pytz'
(myproject)$
```

仮想環境を作ってから、デフォルトのシステムに戻りたければ、deactivate コマンドを用います。 これは、python3 コマンドラインツールの場所を含めて、環境をシステムのデフォルトに戻します。

```
(myproject)$ which python3
/tmp/myproject/bin/python3
(myproject)$ deactivate
$ which python3
/usr/local/bin/python3
```

myproject環境に再び戻って作業したければ、前と同じくmyprojectディレクトリで source bin/activate を実行すればいいのです。

### 依存関係を複製する

仮想環境ができれば、必要に応じてpipを使ってパッケージをインストールしていけます。実際、環境をどこかに複製しておきたいこともあるでしょう。例えば、ワークステーションの開発環境をデータセンターのサーバに複製したいとか、誰かの環境を自分のマシンにクローンして、そのコードのデバッグを手伝うとかです。

venv はそういった状況をたやすく作ることができます。python3 -m pip freeze コマンドを使って、明示的なパッケージ依存関係すべてを (便宜上 requirements.txtという名前の) ファイルに保存できます。


```
(myproject)$ python3 -m pip freeze > requirements.txt
(myproject)$ cat requirements.txt

certifi==2019.3.9
chardet==3.0.4
idna==2.8
numpy==1.16.2
pytz==2018.9
calubom?nt enrgnindex"
s3vq' bemen Jubom of abnuo dollslubol
requests==2.21.0
urllib3==1.24.1
```

myproject に相当する別の仮想環境を作っておきたいとしましょう。 以前と同じく venv を使って新たなディレクトリを作り、それを activate できます。


```
$ python3 -m venv otherproject
$ cd otherproject
$ source bin/activate

(otherproject) $
```

この新環境にはパッケージはまだインストールされていません。


```
(otherproject)$ python3 -m pip list

Package     Version
----------  ---------
pip         18.0.1
setuptools  39.0.1
```

python3 -m pip freeze コマンドで生成した requirements.txt に、python3 -m pip install を実行して、最初の環境にあったすべてのパッケージをインストールすることができます。

```
(otherproject)$ python3 -m pip install -r /tmp/myproject/requirements.txt
```

このコマンドにより、最初の環境を複製するのに必要なパッケージがすべて選択されて順にインストールされます。これができたら、この第二の環境でインストールされたパッケージをリストすれば、 最初の仮想環境と同じ依存関係のリストが得られます。


```
(otherproject)$ python3 -m pip list

Package     Version
----------  ---------
certifi     2019.3.9
(中略)
urllib3 1.24.1
```

requirements.txt ファイルを使うことは、バージョン管理システムを用いて他の人と協働するのに理想的です。コードの変更にコミットすると同時に、パッケージ依存関係のリストを更新して、作業の同期を取って進めることができます。 しかし、使用しているPythonのバージョンが requirements.txt ファイルに含まれていないことに注意することが重要です。そのために、別途管理しないといけません。

仮想環境で理解しないといけないのは、場所を移動すると python3 のようなすべてのパスが環境のインストールディレクトリの中に直接埋め込まれているので、何もかもが壊れてしまうということです。しかし、それは大したことではありません。仮想環境のそもそもの目的は、 同じセットアップを複製するのを容易にすることでした。仮想環境ディレクトリを動かす代わりに、古いのを python3 -m pip freeze して、新しい仮想環境をどこかに作り、requirements.txt ファイルですべてを再イ
ンストールすればいいだけなのですから。

### 覚えておくこと

- 仮想環境は、pip を使って、同じマシンに同じパッケージの異なるバージョンを問題を起こ さずにインストールすることを可能にする
- 仮想環境は、python -m venvで作られ、source bin/activate で利用可能になり、deactivate で停止する
- python-m pip freeze で、環境のすべての要件をダンプすることができる。python -m pip install -r requirements.txtで、環境を複製することができる。


## 項目 84 すべての関数、クラス、モジュールについて docstring を書く


Python におけるドキュメンテーションは、 言語の動的な性質からして非常に重要です。他の多くの言語とは異なり、プログラムのソースコードのドキュメンテーションは、プログラムから実行時に直接アクセスできます。

例えば、 関数のdef文の直後にdocstring (ドキュメンテーション文字列)を与えることによってドキュメンテーションを追加することができます。

In [1]:
# Example 1
def palindrome(word):
    """Return True if the given word is a palindrome."""
    return word == word[::-1]   # 単語が回文ならTrueを返す

assert palindrome('tacocat')
assert not palindrome('banana')

関数の特別な属性__doc__ にアクセスすることでPython プログラムの中からdocstring を取り出せます。

In [2]:
print(repr(palindrome.__doc__))

'Return True if the given word is a palindrome.'


コマンドラインから組み込みモジュール pydocを使って、 Pythonインタプリタからアクセスできる Python ドキュメンテーションをホストするローカルウェブサーバを実行することもできます。

```
$ python3 -m pydoc -p 1234

Server ready at http://localhost: 1234/ 
Server commands: [b]rowser, [q]uit
server > b
```

docstring は、 関数、クラス、モジュールに付与できます。 この関連付けは、 Python プログラムをコンパイルして実行するプロセスの一部です。 docstring と__doc__属性をサポートすることによって、 次の3つの成果が得られています。


- アクセスしやすいドキュメンテーションがあると、 対話的な開発が容易になる。 組み込み関数 helpを用いてドキュメンテーションを読み、関数、クラス、モジュールを調べることができる。 アルゴリズムを開発し、APIをテストし、コードを書くことが、これによって、Python の対話的 インタプリタ (Python Shell) や IPython Notebook (https://ipython.org) のようなツールを使って楽しくできる。
- ドキュメンテーションを定義する標準的な方法を使うと、テキストを (HTMLのような)より魅力的なフォーマットに変換するツールを作りやすい。 これが、 Sphinx (https://www.sphinx-doc.org) のような Python コミュニティのための優れたドキュメンテーション生成ツールにつながっている。さらには、きれいに見えるドキュメンテーション用のオープンソース Pythonプロジェ クトをフリーにホストする Read the Docs (https://readthedocs.org) のようなコミュニティがサポートするサイトを可能にしている。
- Pythonの、ファーストクラスで、アクセス可能で、可読性に優れたドキュメンテーションは、 人々にさらに多くのドキュメンテーション書く気にさせる。 Python コミュニティは、ドキュメン テーションの重要性を強く信じている。「良いコード」がドキュメンテーションの良いコードを意味していると考えられている。 これは、ほとんどのオープンソース Python ライブラリにきちんとしたドキュメンテーションがあると期待できることを意味する。


この優れたドキュメンテーション文化に参加するには、 docstringを書くときにいくつかのガイドラ インに従う必要があります。 詳細は、PEP 257 (https://www.python.org/dev/peps/pep-0257/)でオンラインで議論されています。間違いなく従うべきベストプラクティスがあります。

### モジュールのドキュメンテーション

各モジュールのトップレベルに、すなわちソースファイルの最初に、docstring の文字列リテラルを 置くべきです。3連二重引用符 (""") を使います。このdocstringの目的は、 モジュールとその内容の紹介です。

docstringの第1行は、モジュールの目的を述べる1つの文です。その次の段落は、モジュールの全 ユーザがその働きについて知っておくべきことの詳細です。 モジュール docstringから、モジュールにある重要なクラスや関数を探し出すことができます。

モジュール docstring の例は次の通りです。


```
# words.py
#!/usr/bin/env python3

"""Library for testing words for various linguistic patterns.'※
Testing how words relate to each other can be tricky sometimes!
This module provides easy ways to determine when words you've found have special properties.
Available functions:
palindrome: Determine if a word is a palindrome.
check_anagram: Determine if two words are anagrams.
...
"""
```
※訳注: docstring の訳は次の通り

さまざまな言語処理パターンで単語をテストするライブラリ単語の相互関係のテストは難しいことがある。このモジュールで、 特定の性質を持つかどうかがわかる。

含まれる関数
- palindrome: 単語が回文か調べる
- check_anagram: 2つの単語がアナグラムか調べる


モジュールがコマンドラインユーティリティなら、 モジュール docstring は、コマンドラインからこのツールを利用するための情報を示すのに最適の場所です。

### クラスのドキュメンテーション

どのクラスもクラスレベルの docstring を持つべきです。これは、ほぼ、モジュールレベルの docstring と同じ形式です。第1行は、クラスの目的という1つの文です。続く段落は、クラスの演算の重要なところを詳細に述べます。

クラスの重要なパブリックな属性とメソッドをクラスレベルの docstring で述べます。プロテクテッド属性 (項目42 プライベート属性よりパブリックな属性が好ましい」 参照) やスーパークラスのメソッドを正しく扱うためのサブクラスに対する指針も提供すべきです。

クラスの docstring の例は、次のようになります。


In [3]:
# Example 4
class Player:
    """Represents a player of the game.

    Subclasses may override the 'tick' method to provide
    custom animations for the player's movement depending
    on their power level, etc.

    Public attributes:
    - power: Unused power-ups (float between 0 and 1).
    - coins: Coins found during the level (integer).
    """
    # ...

※訳注:docstringの訳は次の通り

ゲームのプレイヤーを表す。サブクラスはtickメソッドをオーバーライドしてパワーレベルに応じたアニメーションを行う。

パブリックな属性
- power: 未使用パワーアップ (0から1のfloat)
- coins: レベルで見つかったコイン（integer）


### 関数のドキュメンテーション

パブリック関数とメソッドには、docstring を付けるべきです。モジュールやクラスと同じ形式です。第1行は、関数が何をするかを述べる1つの文です。続く段落で、振る舞いと引数について述べます。戻り値があれば述べます。呼び出し元が関数のインタフェースの一部として扱う例外は説明すべきです(「項目20 None を返すのではなく例外を送出する」 参照)。

関数の docstring の例を示します。


In [ ]:
# Example 5
import itertools

def find_anagrams(word, dictionary):
    """Find all anagrams for a word.

    This function only runs as fast as the test for
    membership in the 'dictionary' container.

    Args:
        word: String of the target word.
        dictionary: collections.abc.Container with all
            strings that are known to be actual words.

    Returns:
        List of anagrams that were found. Empty if
        none were found.
    """
    permutations = itertools.permutations(word, len(word))
    possible = (''.join(x) for x in permutations)
    found = {word for word in possible if word in dictionary}
    return list(found)

assert find_anagrams('pancakes', ['scanpeak']) == ['scanpeak']

*2 訳注: docstring の訳は次の通り

単語の全アナグラムを見つける。この関数は dictionary コンテナのメンバー検査の速度で実行する。

引数
- word: 単語の文字列
- dictionary: 単語としてわかっている全文字列を含む
  
戻り値
- 得られたアナグラムのリスト。なければ空


関数の docstring を書く際に知っておくのが重要な次のような特別な場合があります。
- 関数が引数を持たず単純な戻り値を返すなら、1つの文で述べれば十分。
- 関数が何も返さないなら、 「returns None」と書くくらいなら戻り値について何も書かないほうがよい。
- 関数のインタフェースが例外を含むなら (「項目20 None を返すのではなく例外を送出する」 参照) docstring には、 発生しうる例外とそれがどのような場合に起こるかを述べる。
- 関数が通常の演算で例外を起こさないなら、何も書かない。
- 関数が可変個数引数 (「項目22 可変長位置引数を使って、 見た目をすっきりさせる」 参照) や キーワード引数 (項目23 キーワード引数にオプションの振る舞いを与える」参照)を取るなら、docstringの中で引数リストで *args と **kwargsをその目的とともに述べる
- 関数がデフォルト値のある引数を取るなら、 そのデフォルト値について述べる (項目24 動的 なデフォルト引数を指定するときには None と docstringを使う」参照)。
- 関数がジェネレータ (項目30 リストを返さずにジェネレータを返すことを考える」 参照) なら、 docstring でジェネレータが何を yield するか述べる。
- 関数が非同期コルーチン (項目60 コルーチンで高度な並行I/Oを達成する」 参照) なら、 docstring で、いつ実行を停止するかを説明する。


### docstring と型ヒントの使用

現在のPython は、 さまざまな目的のために型ヒントをサポートしています (使用法については「項目 90 バグを回避するために静的解析を検討する」参照)。 型ヒントの情報が、docstring と重なることがあります。例えば、find_anagrams の関数シグネチャに型ヒントを適用したとします。

In [4]:
# Example 6
# Check types in this file with: python -m mypy <path>
from typing import Container, List

def find_anagrams(word: str,
                  dictionary: Container[str]) -> List[str]:
    pass


docstringの中で、引数 word が文字列だと述べる必要は、型ヒントに示されているので不要です。dictionary 引数についても、collections.abc.Container であることを述べる必要はありません。戻り値の型がlistであることも、型ヒントに明示されているので述べる必要がありません。引数がなくても戻り値がlistなので、空listだとわかりますし、docstring で述べる必要はありません。この関数の docstringを短くして記述します。


In [5]:
# Example 7
# Check types in this file with: python -m mypy <path>

from typing import Container, List

def find_anagrams(word: str,
                  dictionary: Container[str]) -> List[str]:
    """Find all anagrams for a word.

    This function only runs as fast as the test for
    membership in the 'dictionary' container.

    Args:
        word: Target word.
        dictionary: All known actual words.

    Returns:
        Anagrams that were found.
    """
    pass

※訳注:docstringの訳は次の通り

単語の全アナグラムを返す。この関数は、'dictionary ' コンテナのメンバーテストの速度で実行。

引数
- word : 目標単語
- dictionary: 既知の単語全体

戻り値
- 見つけたアナグラム


型ヒントと docstring の重複は、インスタンスフィールド、クラス属性、メソッドでも同様にして避けるべきです。型情報は1か所にまとめ、実際の実装と矛盾がないようにすることが大事です。

### 覚えておくこと

- あらゆるモジュール、クラス、メソッド、関数に docstring を使ってドキュメンテーション を書く。コードの変化に追随して、ドキュメンテーションを最新に保つ。
- モジュールについて、 モジュールの内容とすべてのユーザが知っておくべき重要なクラスや 関数を紹介する。
- クラスについて、 class 文に続く docstring に、振る舞い、重要な属性、サブクラスの振 る舞いを記述する。
- 関数とメソッドについて、def 文に続く docstring に、すべての引数、戻り値、引き起こさ れる例外 その他の振る舞いを記述する。
- 型ヒントを使うなら、両方に同じ情報が重複しないように、型ヒントにある情報を docstring には書かない。
